[API documentation](https://dev.elsevier.com/documentation/ScopusSearchAPI.wadl)


In [3]:
import requests
import pandas as pd
import time
import pickle
import re
import json

In [5]:
# to communicate with google spreadsheet...
import gspread
from gspread_dataframe import get_as_dataframe
from gspread_dataframe import set_with_dataframe
from google.oauth2 import service_account # based on google-auth library

# establish connection with gogglesheets...
file_data = json.load(open("../../ServiceAccountsKey.json", "r"))
credentials = service_account.Credentials.from_service_account_info(file_data)
gc = gspread.Client(auth=credentials.with_scopes(['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']))
#qec_gsheet = gc.open_by_url()

In [45]:
apikey = input("go to https://dev.elsevier.com/documentation/ScopusSearchAPI.wadl, generate your API key, copy it and insert it here: ")

In [46]:
headers = {"X-ELS-APIKey" : apikey}

In [66]:
api_address = "https://api.elsevier.com/content/search/scopus?"
base_url = api_address + "&query="
print(base_url)

https://api.elsevier.com/content/search/scopus?&query=


In [67]:
query = "religion"
base_url + query

'https://api.elsevier.com/content/search/scopus?&query=religion'

In [70]:
requests.get(base_url + query, headers=headers).json()["search-results"]["opensearch:totalResults"]

'737269'

In [148]:
#query= "TITLE-ABS-KEY ( \"Paul the Apostle\"  OR  \"Apostle Paul\"  OR  \"Paul of Tarsus\"  OR  \"Pauline liter\" )"
query = "KEY \"adventis*\"AND DOCTYPE(ar)"
resp = requests.get(base_url + query, headers=headers)
resp_json = resp.json()
resp_json["search-results"]["opensearch:totalResults"]

'2523'

In [149]:
items = resp_json["search-results"]["opensearch:totalResults"]
pagelength = resp_json["search-results"]["opensearch:itemsPerPage"]
entries = resp_json["search-results"]["entry"]

In [150]:
indices = list(range(0, int(items), int(pagelength)))
print(indices)

[0, 25, 50, 75, 100, 125, 150, 175, 200, 225, 250, 275, 300, 325, 350, 375, 400, 425, 450, 475, 500, 525, 550, 575, 600, 625, 650, 675, 700, 725, 750, 775, 800, 825, 850, 875, 900, 925, 950, 975, 1000, 1025, 1050, 1075, 1100, 1125, 1150, 1175, 1200, 1225, 1250, 1275, 1300, 1325, 1350, 1375, 1400, 1425, 1450, 1475, 1500, 1525, 1550, 1575, 1600, 1625, 1650, 1675, 1700, 1725, 1750, 1775, 1800, 1825, 1850, 1875, 1900, 1925, 1950, 1975, 2000, 2025, 2050, 2075, 2100, 2125, 2150, 2175, 2200, 2225, 2250, 2275, 2300, 2325, 2350, 2375, 2400, 2425, 2450, 2475, 2500]


In [151]:
base_url + query + "&start={0}".format(str(i))+ "&view=complete"

'https://api.elsevier.com/content/search/scopus?&query=KEY "adventis*"AND DOCTYPE(ar)&start=150&view=complete'

In [152]:
url = 'https://api.elsevier.com/content/search/scopus'
headers = {
    'X-ELS-APIKey': apikey,
    'Accept': 'application/json'
}

params = {
    'query': 'KEY "early christian*"',
    'start': 0,
    'view': 'complete'
}

In [153]:
resp = requests.get(url, headers=headers, params=params)
resp

<Response [200]>

In [154]:
%%time
entries = []
for i in indices:
    resp = requests.get(base_url + query + "&start={0}".format(str(i)) + "&view=complete", headers=headers)
    print(i, resp)
    resp_json = resp.json()
    entry = resp_json["search-results"]["entry"]
    entries.extend(entry)

0 <Response [200]>
25 <Response [200]>
50 <Response [200]>
75 <Response [200]>
100 <Response [200]>
125 <Response [200]>
150 <Response [200]>
175 <Response [200]>
200 <Response [200]>
225 <Response [200]>
250 <Response [200]>
275 <Response [200]>
300 <Response [200]>
325 <Response [200]>
350 <Response [200]>
375 <Response [200]>
400 <Response [200]>
425 <Response [200]>
450 <Response [200]>
475 <Response [200]>
500 <Response [200]>
525 <Response [200]>
550 <Response [200]>
575 <Response [200]>
600 <Response [200]>
625 <Response [200]>
650 <Response [200]>
675 <Response [200]>
700 <Response [200]>
725 <Response [200]>
750 <Response [200]>
775 <Response [200]>
800 <Response [200]>
825 <Response [200]>
850 <Response [200]>
875 <Response [200]>
900 <Response [200]>
925 <Response [200]>
950 <Response [200]>
975 <Response [200]>
1000 <Response [200]>
1025 <Response [200]>
1050 <Response [200]>
1075 <Response [200]>
1100 <Response [200]>
1125 <Response [200]>
1150 <Response [200]>
1175 <Respo

In [155]:
pickle.dump(entries, open("../data/entries_adventis.pkl", "wb"))

# working with raw entries

In [156]:
#entries = pickle.load(open("../data/entries_v1.pkl", "rb"))

In [157]:
entries_df = pd.DataFrame(entries)

In [158]:
entries_df.columns

Index(['@_fa', 'link', 'prism:url', 'dc:identifier', 'eid', 'dc:title',
       'dc:creator', 'prism:publicationName', 'prism:issn', 'prism:eIssn',
       'prism:volume', 'prism:issueIdentifier', 'prism:pageRange',
       'prism:coverDate', 'prism:coverDisplayDate', 'prism:doi',
       'dc:description', 'citedby-count', 'affiliation', 'pubmed-id',
       'prism:aggregationType', 'subtype', 'subtypeDescription',
       'author-count', 'author', 'authkeywords', 'source-id', 'fund-acr',
       'fund-no', 'fund-sponsor', 'openaccess', 'openaccessFlag', 'freetoread',
       'freetoreadLabel', 'pii', 'article-number', 'prism:isbn'],
      dtype='object')

In [159]:
entries_df[entries_df["subtypeDescription"]=="Article"].tail(30)

,@_fa,link,prism:url,dc:identifier,eid,dc:title,dc:creator,prism:publicationName,prism:issn,prism:eIssn,...,fund-acr,fund-no,fund-sponsor,openaccess,openaccessFlag,freetoread,freetoreadLabel,pii,article-number,prism:isbn
2495,true,"[{'@_fa': 'true', '@ref': 'self', '@href': 'ht...",https://api.elsevier.com/content/abstract/scop...,SCOPUS_ID:10444261976,2-s2.0-10444261976,Dietary risk factors for the development of in...,Pattison D.,Arthritis and Rheumatism,00043591,NaN,...,NaN,undefined,NaN,0,False,NaN,NaN,NaN,NaN,NaN
2496,true,"[{'@_fa': 'true', '@ref': 'self', '@href': 'ht...",https://api.elsevier.com/content/abstract/scop...,SCOPUS_ID:12344301273,2-s2.0-12344301273,The role of Vitamin D <inf>3</inf> metabolism ...,Lou Y.R.,Journal of Steroid Biochemistry and Molecular ...,09600760,NaN,...,NaN,undefined,NaN,0,False,NaN,NaN,S0960076004003723,NaN,NaN
2497,true,"[{'@_fa': 'true', '@ref': 'self', '@href': 'ht...",https://api.elsevier.com/content/abstract/scop...,SCOPUS_ID:10844234765,2-s2.0-10844234765,A prospective study of diet and prostate cance...,Allen N.E.,Cancer Causes and Control,09575243,NaN,...,NaN,14-78,National Academy of Sciences,0,False,NaN,NaN,NaN,NaN,NaN
2498,true,"[{'@_fa': 'true', '@ref': 'self', '@href': 'ht...",https://api.elsevier.com/content/abstract/scop...,SCOPUS_ID:5444258710,2-s2.0-5444258710,Health benefits of soy isoflavonoids and strat...,McCue P.,Critical Reviews in Food Science and Nutrition,10408398,NaN,...,NaN,undefined,NaN,0,False,NaN,NaN,NaN,NaN,NaN
2499,true,"[{'@_fa': 'true', '@ref': 'self', '@href': 'ht...",https://api.elsevier.com/content/abstract/scop...,SCOPUS_ID:6344249103,2-s2.0-6344249103,The UK Women's Cohort Study: Comparison of veg...,Cade J.,Public Health Nutrition,13689800,NaN,...,WCRF,undefined,World Cancer Research Fund,1,True,"{'value': [{'$': 'all'}, {'$': 'publisherfree2...","{'value': [{'$': 'All Open Access'}, {'$': 'Br...",NaN,NaN,NaN
2500,true,"[{'@_fa': 'true', '@ref': 'self', '@href': 'ht...",https://api.elsevier.com/content/abstract/scop...,SCOPUS_ID:19944389541,2-s2.0-19944389541,Smad3 mediates TGF-β1 induction of VEGF produc...,Kobayashi T.,Biochemical and Biophysical Research Communica...,0006291X,NaN,...,NIH,R01HL064088,National Institutes of Health,0,False,NaN,NaN,S0006291X04028177,NaN,NaN
2501,true,"[{'@_fa': 'true', '@ref': 'self', '@href': 'ht...",https://api.elsevier.com/content/abstract/scop...,SCOPUS_ID:13944272308,2-s2.0-13944272308,Veganism and its relationship with insulin res...,Goff L.M.,European Journal of Clinical Nutrition,09543007,NaN,...,GSK,undefined,GlaxoSmithKline,0,False,NaN,NaN,NaN,NaN,NaN
2502,true,"[{'@_fa': 'true', '@ref': 'self', '@href': 'ht...",https://api.elsevier.com/content/abstract/scop...,SCOPUS_ID:85009644397,2-s2.0-85009644397,"Refocusing on the learning in ""integration of ...",Lawrence T.A.,Journal of Research on Christian Education,10656219,19344945,...,NaN,undefined,NaN,0,False,NaN,NaN,NaN,NaN,NaN
2503,true,"[{'@_fa': 'true', '@ref': 'self', '@href': 'ht...",https://api.elsevier.com/content/abstract/scop...,SCOPUS_ID:76349090690,2-s2.0-76349090690,Preoperative body mass index and outcomes of g...,Ogoshi K.,Annals of Cancer Research and Therapy,13446835,NaN,...,NaN,undefined,NaN,1,True,"{'value': [{'$': 'all'}, {'$': 'publisherfree2...","{'value': [{'$': 'All Open Access'}, {'$': 'Br...",NaN,NaN,NaN
2504,true,"[{'@_fa': 'true', '@ref': 'self', '@href': 'ht...",https://api.elsevier.com/content/abstract/scop...,SCOPUS_ID:70549087634,2-s2.0-70549087634,Position of the American Dietetic Association ...,NaN,Medicina Naturista,15763080,2386463X,...,NaN,undefined,NaN,0,False,NaN,NaN,NaN,NaN,NaN


In [160]:
entries_df[entries_df["dc:description"].notnull()]["dc:description"].apply(lambda x: x.count("interpret")).sum()

50

In [161]:
entries_df[entries_df["dc:description"].notnull()]["dc:description"].apply(lambda x: x.count("quantitative")).sum()

99

In [ ]:
entries_df["dc:description"].apply(re)

In [100]:
resp = requests.get("https://api.elsevier.com/content/abstract/scopus_id/85034222301%29&view=COMPLETE&field=dc:title,dc:description,prism:doi", headers=headers)
resp

<Response [500]>

In [99]:
resp.json()

{'abstracts-retrieval-response': {'affiliation': [{'affiliation-city': 'Pretoria',
    'affilname': 'University of Pretoria',
    'affiliation-country': 'South Africa'},
   {'affiliation-city': 'Amsterdam',
    'affilname': 'Vrije Universiteit Amsterdam',
    'affiliation-country': 'Netherlands'},
   {'affiliation-city': 'Utrecht',
    'affilname': 'Universiteit Utrecht',
    'affiliation-country': 'Netherlands'}],
  'coredata': {'srctype': 'j',
   'prism:issueIdentifier': '2',
   'eid': '2-s2.0-85034222301',
   'prism:coverDate': '2016-01-01',
   'prism:aggregationType': 'Journal',
   'prism:url': 'https://api.elsevier.com/content/abstract/scopus_id/85034222301',
   'subtypeDescription': 'Article',
   'dc:creator': {'author': [{'ce:given-name': 'Peter-Ben',
      'preferred-name': {'ce:given-name': 'Peter Ben',
       'ce:initials': 'P.B.',
       'ce:surname': 'Smit',
       'ce:indexed-name': 'Smit P.B.'},
      '@seq': '1',
      'ce:initials': 'P.-B.',
      '@_fa': 'true',
      

In [96]:
entries_df.drop(columns=['@_fa', 'link', 'freetoread', 'freetoreadLabel'], inplace=True)

In [99]:
entries_df['author-count'] = entries_df['author-count'].apply(lambda x: int(x["@total"]))

In [105]:
def extract_affiliation_names(x):
    try:
        return [el["affilname"] for el in x]
    except:
        []
entries_df["affiliation_simple"] = entries_df["affiliation"].apply(extract_affiliation_names)

In [106]:
set_with_dataframe(mops_data.add_worksheet("entries_df_v3", 1,1), entries_df)

In [107]:
entries_df.columns

Index(['prism:url', 'dc:identifier', 'eid', 'dc:title', 'dc:creator',
       'prism:publicationName', 'prism:eIssn', 'prism:volume',
       'prism:issueIdentifier', 'prism:pageRange', 'prism:coverDate',
       'prism:coverDisplayDate', 'prism:doi', 'dc:description',
       'citedby-count', 'affiliation', 'prism:aggregationType', 'subtype',
       'subtypeDescription', 'author-count', 'author', 'authkeywords',
       'article-number', 'source-id', 'fund-no', 'openaccess',
       'openaccessFlag', 'prism:issn', 'pii', 'fund-acr', 'fund-sponsor',
       'prism:isbn', 'pubmed-id', 'affiliation_simple'],
      dtype='object')

* 'dc:title' - title
* 'dc:creator' - author
* 'prism:publicationName' - name of the journal or of an volume
* 'prism:coverDate' - date of publication
* 'dc:description' - abstract
* 'subtypeDescription' - type of the entry (i,e, either Article or book chapter etc...)
* 'author-count' - number of authors
* 'author' - nested data containing additional information about all authors -> needs to be explored in more detail
* 'affiliation_simple' - a list of all mentioned affiliations

entries_df.columns


In [ ]:
entries_df.columns


# Simple explorations

In [108]:
entries_df[entries_df["author-count"] > 1]

,prism:url,dc:identifier,eid,dc:title,dc:creator,prism:publicationName,prism:eIssn,prism:volume,prism:issueIdentifier,prism:pageRange,...,fund-no,openaccess,openaccessFlag,prism:issn,pii,fund-acr,fund-sponsor,prism:isbn,pubmed-id,affiliation_simple
10,https://api.elsevier.com/content/abstract/scop...,SCOPUS_ID:85126848614,2-s2.0-85126848614,Discourse-Rhetorical Strategies of Pauline Epi...,Chidume S.N.,SAGE Open,21582440,12,1,None,...,undefined,1,True,NaN,NaN,NaN,NaN,NaN,NaN,[University of Nigeria]
24,https://api.elsevier.com/content/abstract/scop...,SCOPUS_ID:85127816783,2-s2.0-85127816783,Saint Paul’s Thorn in the Flesh: a Dermatologi...,Pascual A.M.,Scientia et Fides,23535636,10,1,9-27,...,undefined,1,True,23007648,NaN,NaN,NaN,NaN,NaN,"[Hospital Universitari de Bellvitge, Universid..."
55,https://api.elsevier.com/content/abstract/scop...,SCOPUS_ID:85099398166,2-s2.0-85099398166,How could the spirit not be secular? Dialogue ...,Rohmer C.,Etudes Theologiques et Religieuses,22729011,95,4,625-644,...,undefined,0,False,00142239,NaN,NaN,NaN,NaN,NaN,[Faculté de Montpellier]
82,https://api.elsevier.com/content/abstract/scop...,SCOPUS_ID:85111726701,2-s2.0-85111726701,"Onesimus, philemon’s runaway slave boy: A brie...",Manus C.U.,HTS Teologiese Studies / Theological Studies,20728050,77,1,None,...,undefined,1,True,02599422,NaN,NaN,NaN,NaN,NaN,"[University of Pretoria, National University o..."
89,https://api.elsevier.com/content/abstract/scop...,SCOPUS_ID:85105631267,2-s2.0-85105631267,Jesus as a socially (ir)responsible innovator:...,Woods C.,International Journal of Public Theology,15697320,15,1,118-142,...,undefined,0,False,18725171,NaN,NaN,NaN,NaN,NaN,"[The University of Auckland Business School, F..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1039,https://api.elsevier.com/content/abstract/scop...,SCOPUS_ID:1542463682,2-s2.0-1542463682,The ethic of caring: the moral response to suf...,Hitchens E.,Christian scholar's review,NaN,23,3,307-317,...,undefined,0,False,00172251,NaN,NaN,NaN,NaN,14628772,[Seattle Pacific University]
1045,https://api.elsevier.com/content/abstract/scop...,SCOPUS_ID:84953959083,2-s2.0-84953959083,Testimonium spiritus sancti—an example of bern...,Bell T.M.M.A.C.,Bijdragen,NaN,53,1,62-72,...,undefined,0,False,00062278,NaN,NaN,NaN,NaN,NaN,"[Universiteit van Amsterdam, Dutch Luther Cons..."
1064,https://api.elsevier.com/content/abstract/scop...,SCOPUS_ID:0020629376,2-s2.0-0020629376,Teaching Humanistic Medicine,Taegtmeyer H.,New England Journal of Medicine,15334406,309,14,860-862,...,undefined,0,False,00284793,NaN,NaN,NaN,NaN,6888478,[University of Texas Health Science Center at ...
1066,https://api.elsevier.com/content/abstract/scop...,SCOPUS_ID:0020051029,2-s2.0-0020051029,Death after trauma involving the maxillofacial...,Zachariades N.,Journal of Maxillofacial Surgery,NaN,10,C,123-125,...,undefined,0,False,03010503,S0301050382800246,NaN,NaN,NaN,6954224,[KAT Hospital]
